In [1]:
import pandas as pd
import numpy as np


In [102]:
# load in all csv in dir.
import glob 
path = r'.' 
all_csv = glob.glob(path + '/*.csv')
# this will be the master frame that holds each CSV eventually
frame = pd.DataFrame()
list_ = []
for file_ in all_csv:
    
    
    df = pd.read_csv(file_, index_col='GHCN ID', header=1)
    
    # remove some columns
    
    df.drop(df.columns[21:], axis=1, inplace=True)
    df.drop(df.columns[5:17], axis=1, inplace=True)
    df.drop(df.columns[[2]], axis=1, inplace=True)
    
    # add state in as a convenience
    state = file_.split('/')[-1:]
    df.insert(1, 'State', state[0][0:2])
    
    # fix the letter values, T is trace, M is missing(?) 
    df[df.columns[5:9]] = df[df.columns[5:9]].replace(['M', 'T'], 0.0)
    
    # adjust col names to dates
    year = '2017'
    new_cols = {day: day[-2:] + '/03/' + year for day in df.columns[5:9]}
    df.rename(columns=new_cols, inplace=True)
    
    # now add this df to main dataframe
    list_.append(df)
    
frame = pd.concat(list_)
frame.head()


['./CT-snowfall-201703.csv', './DE-snowfall-201703.csv', './MA-snowfall-201703.csv', './MD-snowfall-201703.csv', './ME-snowfall-201703.csv', './NH-snowfall-201703.csv', './NJ-snowfall-201703.csv', './NY-snowfall-201703.csv', './PA-snowfall-201703.csv', './RI-snowfall-201703.csv', './VA-snowfall-201703.csv', './VT-snowfall-201703.csv']


,Station Name,State,County,Latitude,Longitude,13/03/2017,14/03/2017,15/03/2017,16/03/2017
GHCN ID,,,,,,,,,
USC00060227,BAKERSVILLE,CT,LITCHFIELD,-73.01,41.84,0.0,4.3,12.2,0
USC00060299,BARKHAMSTED,CT,LITCHFIELD,-72.96,41.92,0,4.0,12.0,0.0
US1CTHR0018,BERLIN 2.4 SSE,CT,HARTFORD,-72.76,41.58,0.0,5.7,18.5,0
USC00060808,BRIDGEPORT-SUCCESS HILL,CT,FAIRFIELD,-73.16,41.20,0.0,4.5,2.6,0.0
US1CTFR0009,BROOKFIELD 3.3 SSE,CT,FAIRFIELD,-73.39,41.44,0.0,4.5,8.7,0


In [100]:
frame.groupby('State').last()

,Station Name,County,Latitude,Longitude,13/03/2017,14/03/2017,15/03/2017,16/03/2017
State,,,,,,,,
CT,WINSTED 2.6 NNW,LITCHFIELD,-73.09,41.96,0.0,0,0,0
DE,WILMINGTON PORTER RES,NEW CASTLE,-75.54,39.77,0,3.0,0.0,0.0
MA,YARMOUTH 2.3 SSE,BARNSTABLE,-70.20,41.68,0.0,0.0,0.0,0.0
MD,WORTON 3.9 NW,KENT,-76.13,39.32,0,0,0,0
ME,WOODLAND,WASHINGTON,-67.40,45.16,0,0,10.0,0
NH,YORK POND,COOS,-71.33,44.50,0,0,0,0
NJ,WOODSTOWN 0.6 NW,SALEM,-75.33,39.66,0.0,0,0,0
NY,YOUNGSTOWN 2 NE,NIAGARA,-79.01,43.27,0.0,4.0,3.5,0.1
PA,YORK SPRINGS 0.7 SE,ADAMS,-77.11,40.00,0.0,11.5,4.6,0.0


In [101]:
frame['State'].value_counts()

NY    348
PA    308
VA    255
NJ    215
MD    139
MA    131
ME    117
NH     96
VT     81
CT     42
DE     24
RI     16
Name: State, dtype: int64

In [106]:
# save and reload to check:
output = 'output/merged.csv'
frame.to_csv(output)
df = pd.read_csv(output)
df.tail()

,GHCN ID,Station Name,State,County,Latitude,Longitude,13/03/2017,14/03/2017,15/03/2017,16/03/2017
1767,US1VTOL0001,WESTFIELD 0.7 WNW,VT,ORLEANS,-72.44,44.89,0.0,0.0,14.5,8.5
1768,US1VTWH0006,WESTMINSTER WEST 0.9 E,VT,WINDHAM,-72.52,43.07,0.0,3.0,18.5,0.0
1769,US1VTCL0018,WHEELOCK 1.6 S,VT,CALEDONIA,-72.09,44.56,0.0,0.0,14.7,1.6
1770,USC00439984,WOODSTOCK,VT,WINDSOR,-72.51,43.63,0.0,0.0,14.0,0.2
1771,USC00439988,WORCESTER 2 W,VT,WASHINGTON,-72.58,44.37,0.0,0.0,15.5,3.5
